In [1]:
import os
from dotenv import load_dotenv
import requests

from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool

load_dotenv()
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")


@tool
def get_current_weather(city: str) -> str:
    """Use this tool to get the current weather for a specific city."""

    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        'q': city,
        'appid': WEATHER_API_KEY,
        'units': 'metric',
        'lang': 'fa'
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        return f"Current weather in {city}: {data['weather'][0]['description']}, Temperature: {data['main']['temp']}°C"

    except requests.exceptions.HTTPError as http_err:
        if response.status_code == 404:
            return f"Unfortunately, the city {city} is not available."
        return f"An HTTP error occurred: {http_err}"


tools = [get_current_weather]

SYSTEM_MESSAGE = """
                You are a helpful assistant whose only job is to provide the current weather for a specific city.
                You MUST use the `get_current_weather` tool to answer the user's questions.
                Provide the final answer to the user in a clear and complete sentence.
                """

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_MESSAGE),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)


def ask_agent(question):
    print(f"user question: {question}")
    response = agent_executor.invoke({"input": question})
    print("Agent response:")
    print(response["output"])

In [6]:
ask_agent("Should I take a jacket if I go to Paris today?")

user question: Should I take a jacket if I go to Paris today?


> Entering new AgentExecutor chain...

Invoking: `get_current_weather` with `{'city': 'Paris'}`


Current weather in Paris: آسمان صاف, Temperature: 19.3°CYou may not need a jacket in Paris today as the current temperature is 19.3°C with clear skies.

> Finished chain.
Agent response:
You may not need a jacket in Paris today as the current temperature is 19.3°C with clear skies.


In [ ]:

ask_agent("What is the current weather in London?")
ask_agent("Tell me the weather in Tehran.")
ask_agent("Should I take a jacket if I go to Paris today?")
ask_agent("I'm planning a walk in New York City, how is it outside?")
ask_agent("Could you check the temperature for Los Angeles?")
ask_agent("What's the weather like in Mexico City?")
ask_agent("How's the weather in the fictional city of Atlantis?")
ask_agent("What is the temperature in Celsius for Berlin right now?")
ask_agent("How hot is it in Dubai?")
ask_agent("What is the weather forecast for Tokyo tomorrow?")